## Exercício PMR 3508 - Costa Rican Household
Neste exercício, faremos a análise da base adult, gerando um classificador **KNN** o nível de vulnerabilidade ou nível de carência de domicílios/famílias na Costa Rica. Utilizaremos a biblioteca pandas, bem como a sci-kit learn. A **validação cruzada** será utilizada como forma de comparar os resultados entre classificadores KNN que usam diferentes valores de K. 

**Autor:** Gabriel Augusto Bianchi Azevedo Ferreira - **NUSP :** 8989404

In [ ]:
#Carregando a base de dados
import pandas as pd
import numpy as np
data = pd.read_csv("../input/train.csv",
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")


In [ ]:
data.head()

In [ ]:
data.shape


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,5))
plt.subplot(121)
data.Target.value_counts().plot(kind="bar")
plt.ylabel("Número de famílias")
plt.xlabel("Classes")
t=plt.title("Distribuição das classes na base de dados")
plt.subplot(122)
(data.Target.value_counts()*100/data.Target.value_counts().sum()).plot(kind="bar")
plt.ylabel("%")
plt.xlabel("Classes")
t=plt.title("Distribuição porcentual das classes na base de dados")


** Como existe uma classe que responde por 62.7% dos dados, esse deveria ser nosso baseline para acurácia **

In [ ]:
ndata = data.dropna()
ndata.shape

** Percebemos que a quantidade de dados baixou de 9557 para 156 após removermos as linhas com Missing Data. Algo não parece bom... Vamos olhar para as colunas com dados faltantes...**

In [ ]:
print("Colunas que possuem missing data:")
print(data.columns[data.isnull().any()])

In [ ]:
#Removendo essas colunas
dropped_data = data.drop(labels=['v2a1', 'v18q1', 'rez_esc', 'meaneduc', 'SQBmeaned'], axis=1)

In [ ]:
dropped_data.shape

** Agora temos 5 features a menos, mas ainda temos 9557 linhas de dados !! ** <br> **Vamos selecionar apenas as colunas de dados numéricos:**

In [ ]:
numeric_data = dropped_data.select_dtypes(include=[np.number])
print("(Linhas, Nro. Features) = ", numeric_data.shape)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

knn = KNeighborsClassifier(n_neighbors=100)
scores = cross_val_score(knn, numeric_data, numeric_data.Target, cv=5)
print("O resultado da validacao cruzada foi (media): ", scores.mean())

** Agora usando apenas os dados numéricos, vamos escolher um K, para o KNN **

In [ ]:
score_list = []
k_list = np.arange(1,501,10)
for k in k_list:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, numeric_data, numeric_data.Target, cv=5)
    score_list.append(scores.mean())




In [ ]:
import matplotlib.pyplot as plt
k_list = np.arange(1,501,10)
plt.plot(k_list,score_list)
plt.xlabel("num_neighbors")
plt.title("Score X num_neighbors")
print("O valor máximo de score é: ", np.stack(score_list).max(), " e o valor de K correspondente é :", k_list[np.argmax(np.stack(score_list))])

** Agora vamos usar todos os dados, incluindo os não-numéricos **

In [ ]:
from sklearn import preprocessing
encoded_data = dropped_data.apply(preprocessing.LabelEncoder().fit_transform)
knn = KNeighborsClassifier(n_neighbors=131)
scores = cross_val_score(knn, encoded_data, encoded_data.Target, cv=5)
print("O resultado da validacao cruzada foi (media): ", scores.mean())

** O uso dos dados não-numéricos, à primeira vista não trouxe melhorais. Precisamos escolher um subconjunto das features. Como as features são muitas , vamos utilizar um método do sklearn para escolher as K melhores colunas, mais representativas**

In [ ]:
from sklearn.feature_selection import SelectKBest
sel = SelectKBest(k=50)
selected = sel.fit_transform(encoded_data.loc[:,encoded_data.columns != "Target"], encoded_data.Target)

** Selecionamos assim, as 50 features mais significativas para o problema. Utilizaremos um K=131, como obtido anteriormente **

In [ ]:
knn = KNeighborsClassifier(n_neighbors=131)
scores = cross_val_score(knn, selected, encoded_data.Target, cv=5)
print("O resultado da validacao cruzada foi (media): ", scores.mean())

**Nesse caso, o resultado foi de 65% de acurácia, utilizando-se o o classificador KNN. Essa acurácia está, portanto superior ao baseline de 62.7% **